In [1]:
import pandas as pd
import numpy as np

In [2]:
mat = pd.read_csv("student-mat.csv", sep=";")
por = pd.read_csv("student-por.csv", sep=";")

In [3]:
mat.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [4]:
boys, girls = len(mat[mat['sex'] == 'M']), len(mat[mat['sex'] == 'F'])
boys, girls

(187, 208)

In [5]:
students_17_year_old = len(mat[(mat['age'] == 17) & mat['G3'].between(10, 15, inclusive = True)])
students_17_year_old

59

In [6]:
boys_from_top_10 = len(mat.iloc[((mat.G1 + mat.G2 + mat.G3) / 3).sort_values(ascending = False).index].head(10).query('sex == "M"'))
boys_from_top_10

6

In [7]:
mean, std = mat[(mat['romantic'] == 'yes') & (mat['internet'] == 'yes')]['absences'].mean(), mat[(mat['romantic'] == 'yes') & (mat['internet'] == 'yes')]['absences'].std()
mean, std

(8.155172413793103, 11.25329203873895)

In [8]:
student_counts_for_each_age = mat.groupby('age').size().reset_index()[0].values.tolist()
student_counts_for_each_age

[82, 104, 98, 82, 24, 3, 1, 1]

In [9]:
father, mother = mat.query('Fjob != "other" and Mjob != "other"')\
    .groupby(['Fjob', 'Mjob']).size()\
    .reset_index(name = 'count')\
    .sort_values(by = 'count', ascending = False).head(1)[['Fjob', 'Mjob']].iloc[0]
father, mother

('services', 'services')

In [10]:
both_present = len(mat.dropna().merge(por.dropna(),\
                                  how = 'inner',\
                                  on = list(set(mat.columns) - set(['G1', 'G2', 'G3'])),\
                                  suffixes = ['_mat', '_por']))
both_present

39

In [11]:
GP_diff, MS_diff = abs(mat[mat['school'] == 'GP']['G3'].mean() - por[por['school'] == 'GP']['G3'].mean()),\
    abs(mat[mat['school'] == 'MS']['G3'].mean() - por[por['school'] == 'MS']['G3'].mean())
GP_diff, MS_diff

(2.0868608045953643, 0.8026163909195851)

In [20]:
corr_mat = mat.select_dtypes(include=np.number).sort_index(axis = 1).corr(method = 'pearson').abs()
top5_correlations = (corr_mat.where(np.triu(np.ones(corr_mat.shape), k=1).astype(np.bool)))\
    .stack()\
    .reset_index()\
    .rename(columns = {0: 'c'})\
    .sort_values(ascending = False, by = 'c')\
    .head(5)\
    .drop(columns = ['c'])\
    .values.tolist()
top5_correlations

[['G2', 'G3'], ['G1', 'G2'], ['G1', 'G3'], ['Dalc', 'Walc'], ['Fedu', 'Medu']]

In [13]:
with open('student_answers.txt', 'w') as file:
    file.write(f"{boys} {girls}\n")
    file.write(f"{students_17_year_old}\n")
    file.write(f"{boys_from_top_10}\n")
    file.write(f"{round(mean, 3)} {round(std, 3)}\n")
    file.write(f"{student_counts_for_each_age}\n")
    file.write(f"{father} {mother}\n")
    file.write(f"{both_present}\n")
    file.write(f"{round(GP_diff, 3)} {round(MS_diff, 3)}\n")
    file.write(f"{top5_correlations}")